In [5]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path

#custom modules
import feature_extraction
import unsw_nb15_dataset

from importlib import reload #allow us to reload custom modules any time we like with modifications
reload(feature_extraction)
reload(unswnb15_dataset)

fe.test_cases()

ModuleNotFoundError: No module named 'unsw_nb15_dataset'

In [ ]:
#first, load the dataset
data_set = unsw_nb15_dataset.UNSWNB15(['/home/jaywalker/MachineLearning/PacketGAN/UNSW-NB15_1_clean.csv'])

#pick out the attack examples
data_set.use_only_category('DoS') #in this case just look at DoS attack samples

#convert attack examples into time windows for training


#convert each window into appropriate feature tensor

#associate a "masking" vector with each window

#train GAN to replace masked packets

#observe results